# Section 2: Analyzing Causal Forests on Simulated Data similar to GSS data 

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from econml.dml import CausalForestDML

from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

def fullDisplay():
    pd.set_option("display.max_rows", None, "display.max_columns", None)

def defaultDisplay():
    pd.reset_option('^display.', silent=True)

## Loading data

In [2]:
welfare = pd.read_csv("Data/welfare_clean.csv", low_memory=False)
treatments = welfare['w']
labels = welfare['y']
welfare.drop(columns=['w', 'y'], inplace=True)
welfare

,year,id,wrkstat,hrs1,hrs2,evwork,occ,prestige,wrkslf,wrkgovt,...,adults_miss,unrelat_miss,earnrs_miss,income_miss,rincome_miss,income86_miss,partyid_miss,polviews_miss,attblack,attblack_miss
0,0,1,7,0.004845,0.005228,1,135,0.005641,2,2,...,0,0,0,0,0,0,0,0,0.005440,0
1,0,2,1,0.005055,0.005228,0,106,0.006538,2,2,...,0,1,0,0,1,0,0,0,0.004080,0
2,0,3,7,0.004845,0.005228,1,99,0.006538,2,2,...,0,1,0,0,0,0,0,0,0.002040,0
3,0,4,3,0.005055,0.005228,0,142,0.004615,2,0,...,0,0,0,0,1,0,0,0,0.004080,0
4,0,5,8,0.005055,0.005228,1,211,0.005171,2,1,...,0,0,0,0,0,0,0,0,0.004080,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36496,15,2040,3,0.005055,0.005228,0,211,0.005171,2,2,...,0,0,0,0,1,1,0,0,0.004080,0
36497,15,2041,3,0.005055,0.005228,0,211,0.005171,2,2,...,0,0,0,0,1,1,0,0,0.006120,0
36498,15,2042,7,0.004845,0.005228,1,211,0.005171,2,2,...,0,1,0,0,0,1,0,0,0.004080,0
36499,15,2043,7,0.005935,0.005228,1,211,0.005171,2,2,...,0,1,0,0,1,1,0,0,0.005021,1


## DGP, Estimation, and Evaluation functions

In [18]:
def dgp(welfare, effect_type="heterogeneous", effect_homogeneous=10, effect_heterogeneous=2,
        treatment_type="binary", treatment_probability=0.5, heterogeneous_select=4, overlap=True,
        overlap_percent=0.5, order=3, linearity="med", N=5000):    
    
    featureNames = list(welfare.columns)

    # Define non-confounders
    importantFeatureNames = ['wrkstat', 'race', 'year', 'hrs1', 'income', 'occ80', 'id', 'educ'] # top 8 important features based on Shapley visualization from 
    importantFeatureIndices = []
    for name in importantFeatureNames:
        importantFeatureIndices.append(featureNames.index(name)) 

    # Error terms
    error = np.random.normal(size=(N,1))

    # Data generation
    cov = welfare.cov()
    means = welfare.mean(axis=0)
    X = np.random.multivariate_normal(means.values, cov, size=N, check_valid='warn', tol=1e-8)

    # Linearity specification
    if linearity != 'full':
        select = 0 # select n most important features for interactions and polynomials
        poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)

        if linearity == "high": 
            select = 2
        elif linearity == "med": 
            select = 4
        elif linearity == "low": 
            select = 8
        else: # if some typo, assume baseline of high
            select = 2

        poly.fit(X[:, importantFeatureIndices[:select]])
        fullData = poly.transform(X[:, importantFeatureIndices[:select]])
        fullNames = poly.get_feature_names(input_features=importantFeatureNames[:select])
        higherData = fullData[:, -(fullData.shape[1] - select):] # select only higher order
        higherNames = fullNames[-(len(fullNames) - select):]

        X = np.append(X, higherData, axis=1) 
        featureNames.extend(list(higherNames))

    # Treatment selection
    if treatment_type == "binary":
        # randomly assigned treatments with propensity treatment_probability
        treatments = np.random.choice([0, 1], size=N, p=[1 - treatment_probability, treatment_probability]).reshape((-1, 1))
        if not overlap:
            forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
            treatments[forced] = 0
        treated = treatments > 0

        # generate counterfactual treatment indicator vector
        c_treatments = deepcopy(treatments)
        c_treatments[treatments == 0] = 1
        c_treatments[treatments == 1] = 0
    else:
        treatments = np.random.uniform(size=(N, 1))
        if not overlap:
            forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
            treatments[forced] = 0
        treated = treatments > 0.5

        # to-do: counterfactual treatment vector
        c_treatments = deepcopy(treatments)
        c_treatments[treatments > 0] = 0
        c_treatments[treatments == 0] = treatments.mean()

    # Treatment effect calculation
    if effect_type == "homogeneous":

        T = treatments*effect_homogeneous
        CT = c_treatments*effect_homogeneous

    else: 
        # heterogeneous treatment is effect 1 + effect * (sum of first heterogeneous_select important variables)
        heterogeneousIndices = importantFeatureIndices[:heterogeneous_select]

        T = 1 + (effect_heterogeneous*(X[:, heterogeneousIndices].sum(axis=1)))*treatments.ravel()
        T = T.reshape(-1, 1)
        T[~treated] = np.zeros((~treated).sum())
        
        # counterfactual effect
        CT = 1 + (effect_heterogeneous*(X[:, heterogeneousIndices].sum(axis=1)))*c_treatments.ravel()
        CT = CT.reshape(-1, 1)
        CT[treated] = np.zeros((treated).sum()) # everyone that was treated goes to untreated in counterfactual

    # Outcome calculation
    betas = np.random.normal(size=X.shape[1]).reshape(-1,1)
    y = T + X@betas + error
    cy = CT + X@betas + error
    
    # ATE calculation
    empirical_treated = y[treated] - cy[treated] # the treated (in y)
    empirical_untreated = cy[~treated] - y[~treated] # the untreated
    ate = (empirical_treated.mean() + empirical_untreated.mean())/2
    
    return y, X, betas, featureNames, treatments, cy, ate

def estimate_cf(y, X, treatments, test_size=0.2, criterion='mse', cv=5):
    # split data into train and test sets 
    X_train, X_test, Y_train, Y_test, T_train, T_test = train_test_split(X, y, treatments, test_size=test_size)
        
    # specify hyperparams of model
    est = CausalForestDML(criterion='het', 
                            n_estimators=1000,       
                            max_samples=0.5,
                            discrete_treatment=True,
                            honest=True,
                            inference=True,
                            cv=cv,
                            )
    # fit model
    est.fit(Y_train, T_train, X=X_train, W=None)
        
    return est, X_test

In [13]:
# compute evaluation stats over K simulations
def evalStats(estimators, trueATES, evalSets):
    K = len(estimators)
    
    totalBias = 0
    totalRMSE = 0
    totalCoverage = 0
    totalInterval = 0

    for i in range(K):
        estimateATE = estimators[i].ate(evalSets[i])
        lb, ub = estimators[i].ate_interval(evalSets[i])
        
        bias = trueATES[i] - estimateATE
        totalBias += bias
        
        rmse = bias**2
        totalRMSE += rmse

        coverage = 1 if (trueATES[i] >= lb and trueATES[i] <= ub) else 0
        totalCoverage += coverage

        interval = ub - lb
        totalInterval += interval
    
    return totalBias / K, (totalRMSE / K)**0.5, totalCoverage / K, totalInterval / K

In [17]:
K = 10

## (1) Sample Size

In [19]:
Ns = [1000, 5000, 10000]

biasNs = []
rmseNs = []
coverageNs = []
intervalNs = []

for N in Ns:
    estNs = []
    trueATENs = []
    evalSetNs = []
    for i in range(K):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, N=N)
        est, X_test = estimate_cf(y, X, treatments)

        estNs.append(est)
        trueATENs.append(est.ate(X_test)[0])
        evalSetNs.append(X_test)

    bias, rmse, coverage, interval = evalStats(estNs, trueATENs, evalSetNs)
    
    biasNs.append(bias)
    rmseNs.append(rmse)
    coverageNs.append(coverage)
    intervalNs.append(interval)

KeyboardInterrupt: 

## (2) Degree of non-linearity in X

In [ ]:
linearityLevels = ['full', 'high', 'med', 'low']

biasLLs = []
rmseLLs = []
coverageLLs = []
intervalLLs = []

for level in linearityLevels:
    estLLs = []
    trueATELLs = []
    evalSetLLs = []
    for i in range(K):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, linearity=level)
        est, X_test = estimate_cf(y, X, treatments)

        estLLs.append(est)
        trueATELLs.append(est.ate(X_test)[0])
        evalSetLLs.append(X_test)

    bias, rmse, coverage, interval = evalStats(estLLs, trueATELLs, evalSetLLs)
    
    biasLLs.append(bias)
    rmseLLs.append(rmse)
    coverageLLs.append(coverage)
    intervalLLs.append(interval)

## (3) Percentage Treated

In [ ]:
percentTreats = [0.1, 0.5, 0.9, 1]

biasPTs = []
rmsePTs = []
coveragePTs = []
intervalPTs = []

for percent in percentTreats:
    estPTs = []
    trueATEPTs = []
    evalSetPTs = []
    for i in range(K):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, treatment_probability=percent)
        est, X_test = estimate_cf(y, X, treatments)

        estPTs.append(est)
        trueATEPTs.append(est.ate(X_test)[0])
        evalSetPTs.append(X_test)

    bias, rmse, coverage, interval = evalStats(estPTs, trueATEPTs, evalSetPTs)
    
    biasPTs.append(bias)
    rmsePTs.append(rmse)
    coveragePTs.append(coverage)
    intervalPTs.append(interval)

## (4) Overlap

In [ ]:
overlapTypes = [True, False]

biasOTs = []
rmseOTs = []
coverageOTs = []
intervalOTs = []

for overlap in overlapTypes:
    estOTs = []
    trueATEOTs = []
    evalSetOTs = []
    for i in range(K):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, overlap=overlap)
        est, X_test = estimate_cf(y, X, treatments)

        estOTs.append(est)
        trueATEOTs.append(est.ate(X_test)[0])
        evalSetOTs.append(X_test)

    bias, rmse, coverage, interval = evalStats(estOTs, trueATEOTs, evalSetOTs)
    
    biasOTs.append(bias)
    rmseOTs.append(rmse)
    coverageOTs.append(coverage)
    intervalOTs.append(interval)

## (5) Treatment Effect Heterogeneity Level

In [ ]:
heterogeneityLevels = [0, 2, 4, 8]

biasHLs = []
rmseHLs = []
coverageHLs = []
intervalHLs = []

for heterogeneity in heterogeneityLevels:
    estHLs = []
    trueATEHLs = []
    evalSetHLs = []
    for i in range(K):
        if heterogeneity == 0:
            y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, effect_type='homogeneous')
        else:
            y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, heterogeneous_select=heterogeneity)

        est, X_test = estimate_cf(y, X, treatments)

        estHLs.append(est)
        trueATEHLs.append(est.ate(X_test)[0])
        evalSetHLs.append(X_test)

    bias, rmse, coverage, interval = evalStats(estHLs, trueATEHLs, evalSetHLs)
    
    biasHLs.append(bias)
    rmseHLs.append(rmse)
    coverageHLs.append(coverage)
    intervalHLs.append(interval)

## (6) Treatment Type (Continuous vs. Discrete)

In [ ]:
treatmentTypes = ['binary', 'continuous']

biasTTs = []
rmseTTs = []
coverageTTs = []
intervalTTs = []

for treatment_type in treatmentTypes:
    estTTs = []
    trueATETTs = []
    evalSetTTs = []
    for i in range(K):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, treatment_type=treatment_type)
        est, X_test = estimate_cf(y, X, treatments)

        estTTs.append(est)
        trueATETTs.append(est.ate(X_test)[0])
        evalSetTTs.append(X_test)

    bias, rmse, coverage, interval = evalStats(estTTs, trueATETTs, evalSetTTs)
    
    biasTTs.append(bias)
    rmseTTs.append(rmse)
    coverageTTs.append(coverage)
    intervalTTs.append(interval)

## (7) Alignment

Below is to implement / have running state of variables

In [10]:
np.random.seed(123)
effect_type="heterogeneous"
effect_homogeneous=10
effect_heterogeneous=2
# treatment_type="continuous"
treatment_type="binary"
treatment_probability=0.5
heterogeneous_select=4
order=3
linearity="med"
N=1000
overlap=True
overlap_percent=0.5

featureNames = list(welfare.columns)

importantFeatureNames = ['wrkstat', 'race', 'year', 'hrs1', 'income', 'occ80', 'id', 'educ'] # top 8 important features based on Shapley visualization from 
importantFeatureIndices = []
for name in importantFeatureNames:
    importantFeatureIndices.append(featureNames.index(name)) 

# error terms
error = np.random.normal(size=(N,1))

# data generation
cov = welfare.cov()
means = welfare.mean(axis=0)
X = np.random.multivariate_normal(means.values, cov, size=N, check_valid='warn', tol=1e-8)

# linearity specification
if linearity != 'full':
    select = 0 # select n most important features for interactions and polynomials
    poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)

    if linearity == "high": 
        select = 2
    elif linearity == "med": 
        select = 4
    elif linearity == "low": 
        select = 8
    else: # if some typo, assume baseline of high
        select = 2

    poly.fit(X[:, importantFeatureIndices[:select]])
    fullData = poly.transform(X[:, importantFeatureIndices[:select]])
    fullNames = poly.get_feature_names(input_features=importantFeatureNames[:select])
    higherData = fullData[:, -(fullData.shape[1] - select):] # select only higher order
    higherNames = fullNames[-(len(fullNames) - select):]

    X = np.append(X, higherData, axis=1) 
    featureNames.extend(list(higherNames))

# treatment type
if treatment_type == "binary":
    # randomly assigned treatments with propensity treatment_probability
    treatments = np.random.choice([0, 1], size=N, p=[1 - treatment_probability, treatment_probability]).reshape((-1, 1))
    if not overlap:
        forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
        treatments[forced] = 0
    treated = treatments > 0
    
    # generate counterfactual treatment indicator vector
    c_treatments = deepcopy(treatments)
    c_treatments[treatments == 0] = 1
    c_treatments[treatments == 1] = 0

else:
    treatments = np.random.uniform(size=(N, 1))
    if not overlap:
        forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
        treatments[forced] = 0
    treated = treatments > 0.5
    c_treatments = deepcopy(treatments)
    # to-do
    c_treatments[treatments > 0] = 0
    c_treatments[treatments == 0] = treatments.mean()
    
# treatment effect
if effect_type == "homogeneous":

    T = treatments*effect_homogeneous
    CT = c_treatments*effect_homogeneous
    
else: # heterogeneous
    # heterogeneous treatment is effect 1 + effect * (sum of first heterogeneous_select important variables)
    heterogeneousIndices = importantFeatureIndices[:heterogeneous_select]
    
    T = 1 + (2*(X[:, heterogeneousIndices].sum(axis=1)))*treatments.ravel()
    T = T.reshape(-1, 1)
    T[~treated] = np.zeros((~treated).sum())
    
    CT = 1 + (2*(X[:, heterogeneousIndices].sum(axis=1)))*c_treatments.ravel()
    CT = CT.reshape(-1, 1)
    CT[treated] = np.zeros((treated).sum()) # everyone that was treated goes to untreated in counterfactual
    

betas = np.random.normal(size=X.shape[1]).reshape(-1,1)
y = T + X@betas + error
cy = CT + X@betas + error

In [184]:
X[:, heterogeneousIndices].mean(axis=0).sum() # average across the used columns to check ATE

15.720148792484203

In [176]:
tmp = y[treated] - cy[treated] # the treated
untmp = cy[~treated] - y[~treated] # the untreated
ate = (tmp.mean() + untmp.mean())/2

32.43845628547201

In [11]:
y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, N=36501, order=3, treatment_probability=0.5)

In [13]:
est, X_test = estimate_cf(y, X, treatments)